In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
from model_arch.run_train import *
from utils.step_sample import create_named_schedule_sampler
from model_arch.train import TrainLoop
from utils.data import load_data_text
from model_arch.tokenizer import load_tokenizer, load_model_emb
from model_arch.sampling import sampling

from transformers import AutoTokenizer, PreTrainedTokenizerFast, BertTokenizerFast, set_seed
import json, torch
from utils import dist_util
from functools import partial
import pickle
import random
from datetime import datetime

2024-03-03 16:05:11.654826: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 16:05:11.654903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 16:05:11.665882: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-03 16:05:11.683628: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-03 16:05:13.463478: W tensorflow/compiler/tf2

In [3]:
dist_util.clear_cache()

In [4]:
lr=0.0001
batch_size=50
microbatch=10
epochs=30_000
eval_interval=50
ema_rate='0.9999' 
schedule_sampler='uniform'
diffusion_steps=2000
noise_schedule='sqrt'
vocab='custom'
use_plm_init='no' # embedding in transformer
vocab_size=0
config_name='bert-base-uncased'
seq_len=128
hidden_t_dim=128
hidden_dim=128
dropout=0.15
seed=102
weight_decay=0.0
predict_xstart=True
rescale_timesteps=True
emb_scale_factor=1.0

In [5]:
regular_data_dir='data'
data_player_dir='data/with_player'

# set the data directory
data_dir=regular_data_dir

In [6]:
set_seed(seed)

In [7]:
tokenizer = load_tokenizer('shakespeare', config_name)

shakespeare


In [8]:
model_weight, tokenizer = load_model_emb(hidden_dim, tokenizer)

In [9]:
model_weight

Embedding(30268, 128)

In [10]:
## very very important to set this!!!!!
vocab_size = tokenizer.vocab_size
vocab_size

30268

In [11]:
data = load_data_text(
        batch_size=batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        model_emb=model_weight # use model's weights as init
    )

val = load_data_text(
        batch_size=batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        split='valid',
        model_emb=model_weight, # use model's weights as init
    )

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading form the TRAIN set...
### Data samples...
 ['o hell! what have we here? a carrion death, within whose empty eye there is a written scroll!', 'and his disciples only envy at, ye blew the fire that burns ye now have at ye! enter king,'] ["i'll read the writing. all that glitters is not gold, often have you heard that told", 'frowning on them, takes his seat']
RAM used: 710.78 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 48627
})
RAM used: 729.76 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/48627 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 48627
})
### tokenized_datasets...example [2, 37, 1300, 6, 164, 150, 133, 237, 22, 23, 7135, 432, 10, 906, 569, 3066, 756, 210, 121, 23, 4180, 7422, 6, 3]
RAM used: 775.81 MB


merge and mask:   0%|          | 0/48627 [00:00<?, ? examples/s]

RAM used: 809.20 MB


...

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 12147
})
### tokenized_datasets...example [2, 3886, 121, 105, 520, 10, 2546, 9, 41, 478, 10, 23, 211, 254, 1233, 9840, 187, 4043, 12, 3]
RAM used: 872.35 MB


merge and mask:   0%|          | 0/12147 [00:00<?, ? examples/s]

RAM used: 880.95 MB


padding:   0%|          | 0/12147 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 12147
}) padded dataset
RAM used: 901.86 MB
RAM used: 901.86 MB


In [12]:
model, diffusion = create_model_and_diffusion(
                        hidden_t_dim,
                        hidden_dim,
                        vocab_size,
                        config_name,
                        use_plm_init,
                        dropout,
                        diffusion_steps,
                        noise_schedule,
                        predict_xstart,
                        rescale_timesteps,
                    )

model.to(dist_util.dev())

TransformerNetModel(
  (word_embedding): Embedding(30268, 128)
  (lm_head): Linear(in_features=128, out_features=30268, bias=True)
  (time_embed): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=True)
    (1): SiLU()
    (2): Linear(in_features=512, out_features=768, bias=True)
  )
  (input_up_proj): Sequential(
    (0): Linear(in_features=128, out_features=768, bias=True)
    (1): Tanh()
    (2): Linear(in_features=768, out_features=768, bias=True)
  )
  (input_transformers): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768,

In [13]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

91192508

In [ ]:
schedule_sampler = create_named_schedule_sampler('uniform', diffusion)

train_loss, val_loss = TrainLoop(
                            model=model,
                            diffusion=diffusion,
                            data=data,
                            batch_size=batch_size,
                            microbatch=microbatch,
                            lr=lr,
                            ema_rate=ema_rate,
                            schedule_sampler=schedule_sampler,
                            weight_decay=weight_decay,
                            epochs=epochs,
                            eval_data=val,
                            eval_interval=eval_interval,
                            warm_up_steps=500,
                            use_llrd=True,
                            llrd_rate=0.99
                        ).run_loop()



======== Using Layer-wise Learning Rate Decay with AdamW ========


name: word_embedding.weight, lr: 0.0001
name: lm_head.bias, lr: 0.0001
name: time_embed.0.weight, lr: 0.0001
name: time_embed.0.bias, lr: 0.0001
name: time_embed.2.weight, lr: 0.0001
name: time_embed.2.bias, lr: 0.0001
name: input_up_proj.0.weight, lr: 0.0001
name: input_up_proj.0.bias, lr: 0.0001
name: input_up_proj.2.weight, lr: 0.0001
name: input_up_proj.2.bias, lr: 0.0001
name: input_transformers.layer.0.attention.self.query.weight, lr: 0.00010101010101010101
name: input_transformers.layer.0.attention.self.query.bias, lr: 0.00010101010101010101
name: input_transformers.layer.0.attention.self.key.weight, lr: 0.00010101010101010101
name: input_transformers.layer.0.attention.self.key.bias, lr: 0.00010101010101010101
name: input_transformers.layer.0.attention.self.value.weight, lr: 0.00010101010101010101
name: input_transformers.layer.0.attention.self.value.bias, lr: 0.00010101010101010101
name: input_transformers.la

Epoch 0/30000 Training Loss: 1.1970067024230957
Epoch 0/30000 Validation Loss: 1.1958271265029907
Cleared directory to save new best model.
============>Saving current best model with min_val_loss=1.1958271265029907<=============
Epoch 1/30000 Training Loss: 1.2004178762435913
Epoch 2/30000 Training Loss: 1.1941783428192139
Epoch 3/30000 Training Loss: 1.1934547424316406
Epoch 4/30000 Training Loss: 1.1885653734207153
Epoch 5/30000 Training Loss: 1.181810736656189
Epoch 6/30000 Training Loss: 1.1719131469726562
Epoch 7/30000 Training Loss: 1.1592326164245605
Epoch 8/30000 Training Loss: 1.149044394493103
Epoch 9/30000 Training Loss: 1.1384356021881104
Epoch 10/30000 Training Loss: 1.1222847700119019
Epoch 11/30000 Training Loss: 1.1068856716156006
Epoch 12/30000 Training Loss: 1.0870779752731323
Epoch 13/30000 Training Loss: 1.0685601234436035
Epoch 14/30000 Training Loss: 1.051645040512085
Epoch 15/30000 Training Loss: 1.0343204736709595
Epoch 16/30000 Training Loss: 1.007979154586792

Epoch 152/30000 Training Loss: 0.29737505316734314
Epoch 153/30000 Training Loss: 0.31367024779319763
Epoch 154/30000 Training Loss: 0.2956079840660095
Epoch 155/30000 Training Loss: 0.2985587418079376
Epoch 156/30000 Training Loss: 0.2929301857948303
Epoch 157/30000 Training Loss: 0.29070812463760376
Epoch 158/30000 Training Loss: 0.27864888310432434
Epoch 159/30000 Training Loss: 0.3140440583229065
Epoch 160/30000 Training Loss: 0.29640060663223267
Epoch 161/30000 Training Loss: 0.29143720865249634
Epoch 162/30000 Training Loss: 0.31726202368736267
Epoch 163/30000 Training Loss: 0.30768561363220215
Epoch 164/30000 Training Loss: 0.29375940561294556
Epoch 165/30000 Training Loss: 0.2734200358390808
Epoch 166/30000 Training Loss: 0.285648375749588
Epoch 167/30000 Training Loss: 0.29704052209854126
Epoch 168/30000 Training Loss: 0.2962569296360016
Epoch 169/30000 Training Loss: 0.3092721402645111
Epoch 170/30000 Training Loss: 0.3044865131378174
Epoch 171/30000 Training Loss: 0.31100806

Epoch 304/30000 Training Loss: 0.19740399718284607
Epoch 305/30000 Training Loss: 0.19297859072685242
Epoch 306/30000 Training Loss: 0.1779545247554779
Epoch 307/30000 Training Loss: 0.18243622779846191
Epoch 308/30000 Training Loss: 0.20063288509845734
Epoch 309/30000 Training Loss: 0.1868320256471634
Epoch 310/30000 Training Loss: 0.1787230223417282
Epoch 311/30000 Training Loss: 0.1886361837387085
Epoch 312/30000 Training Loss: 0.18657103180885315
Epoch 313/30000 Training Loss: 0.17386110126972198
Epoch 314/30000 Training Loss: 0.17631809413433075
Epoch 315/30000 Training Loss: 0.18970216810703278
Epoch 316/30000 Training Loss: 0.19067072868347168
Epoch 317/30000 Training Loss: 0.2004091739654541
Epoch 318/30000 Training Loss: 0.189642071723938
Epoch 319/30000 Training Loss: 0.1978829801082611
Epoch 320/30000 Training Loss: 0.18722756206989288
Epoch 321/30000 Training Loss: 0.1832505762577057
Epoch 322/30000 Training Loss: 0.18679013848304749
Epoch 323/30000 Training Loss: 0.1923277

Epoch 455/30000 Training Loss: 0.13888239860534668
Epoch 456/30000 Training Loss: 0.13394096493721008
Epoch 457/30000 Training Loss: 0.14087611436843872
Epoch 458/30000 Training Loss: 0.1354662925004959
Epoch 459/30000 Training Loss: 0.1427529901266098
Epoch 460/30000 Training Loss: 0.13249556720256805
Epoch 461/30000 Training Loss: 0.13626427948474884
Epoch 462/30000 Training Loss: 0.1332254409790039
Epoch 463/30000 Training Loss: 0.13597211241722107
Epoch 464/30000 Training Loss: 0.1355876922607422
Epoch 465/30000 Training Loss: 0.14648839831352234
Epoch 466/30000 Training Loss: 0.133464053273201
Epoch 467/30000 Training Loss: 0.13944238424301147
Epoch 468/30000 Training Loss: 0.12180743366479874
Epoch 469/30000 Training Loss: 0.13645844161510468
Epoch 470/30000 Training Loss: 0.12757885456085205
Epoch 471/30000 Training Loss: 0.11784116923809052
Epoch 472/30000 Training Loss: 0.12752962112426758
Epoch 473/30000 Training Loss: 0.1366899013519287
Epoch 474/30000 Training Loss: 0.13908

...

Epoch 16936/30000 Training Loss: 0.039639078080654144
Epoch 16937/30000 Training Loss: 0.041909605264663696
Epoch 16938/30000 Training Loss: 0.0371256098151207
Epoch 16939/30000 Training Loss: 0.0433310940861702
Epoch 16940/30000 Training Loss: 0.042295582592487335
Epoch 16941/30000 Training Loss: 0.03768568113446236
Epoch 16942/30000 Training Loss: 0.0477200448513031
Epoch 16943/30000 Training Loss: 0.04092716425657272
Epoch 16944/30000 Training Loss: 0.044743575155735016
Epoch 16945/30000 Training Loss: 0.039907798171043396
Epoch 16946/30000 Training Loss: 0.04028746858239174
Epoch 16947/30000 Training Loss: 0.042080480605363846
Epoch 16948/30000 Training Loss: 0.04114952310919762
Epoch 16949/30000 Training Loss: 0.04679793119430542
Epoch 16950/30000 Training Loss: 0.04378896951675415
Epoch 16950/30000 Validation Loss: 0.03808325156569481
Epoch 16951/30000 Training Loss: 0.04524725303053856
Epoch 16952/30000 Training Loss: 0.04008036106824875
Epoch 16953/30000 Training Loss: 0.040088

Epoch 17087/30000 Training Loss: 0.04431139677762985
Epoch 17088/30000 Training Loss: 0.040849719196558
Epoch 17089/30000 Training Loss: 0.05037592723965645
Epoch 17090/30000 Training Loss: 0.0368606373667717
Epoch 17091/30000 Training Loss: 0.04077469930052757
Epoch 17092/30000 Training Loss: 0.040293049067258835
Epoch 17093/30000 Training Loss: 0.03743733465671539
Epoch 17094/30000 Training Loss: 0.03901582956314087
Epoch 17095/30000 Training Loss: 0.04588506743311882
Epoch 17096/30000 Training Loss: 0.03893381357192993
Epoch 17097/30000 Training Loss: 0.04602644965052605
Epoch 17098/30000 Training Loss: 0.04355207085609436
Epoch 17099/30000 Training Loss: 0.04618249461054802
Epoch 17100/30000 Training Loss: 0.03608442470431328
Epoch 17100/30000 Validation Loss: 0.038836706429719925
Epoch 17101/30000 Training Loss: 0.04530809447169304
Epoch 17102/30000 Training Loss: 0.04808112233877182
Epoch 17103/30000 Training Loss: 0.04342027008533478
Epoch 17104/30000 Training Loss: 0.0444552227

Epoch 17238/30000 Training Loss: 0.0447341687977314
Epoch 17239/30000 Training Loss: 0.039657462388277054
Epoch 17240/30000 Training Loss: 0.0414966456592083
Epoch 17241/30000 Training Loss: 0.037363048642873764
Epoch 17242/30000 Training Loss: 0.0368565209209919
Epoch 17243/30000 Training Loss: 0.03618202358484268
Epoch 17244/30000 Training Loss: 0.03946944326162338
Epoch 17245/30000 Training Loss: 0.04213006794452667
Epoch 17246/30000 Training Loss: 0.05154275894165039
Epoch 17247/30000 Training Loss: 0.037897739559412
Epoch 17248/30000 Training Loss: 0.0464872345328331
Epoch 17249/30000 Training Loss: 0.04009057953953743
Epoch 17250/30000 Training Loss: 0.03615758568048477
Epoch 17250/30000 Validation Loss: 0.04539017006754875
Epoch 17251/30000 Training Loss: 0.048921700567007065
Epoch 17252/30000 Training Loss: 0.0400889627635479
Epoch 17253/30000 Training Loss: 0.04907383769750595
Epoch 17254/30000 Training Loss: 0.03767434507608414
Epoch 17255/30000 Training Loss: 0.0457570850849

Epoch 17389/30000 Training Loss: 0.04652409628033638
Epoch 17390/30000 Training Loss: 0.041155971586704254
Epoch 17391/30000 Training Loss: 0.03764873743057251
Epoch 17392/30000 Training Loss: 0.03909387066960335
Epoch 17393/30000 Training Loss: 0.037576574832201004
Epoch 17394/30000 Training Loss: 0.044741805642843246
Epoch 17395/30000 Training Loss: 0.039035238325595856
Epoch 17396/30000 Training Loss: 0.04669235646724701
Epoch 17397/30000 Training Loss: 0.034100279211997986
Epoch 17398/30000 Training Loss: 0.03852841630578041
Epoch 17399/30000 Training Loss: 0.03717251121997833
Epoch 17400/30000 Training Loss: 0.042990751564502716
Epoch 17400/30000 Validation Loss: 0.04158739373087883
Epoch 17401/30000 Training Loss: 0.0473649799823761
Epoch 17402/30000 Training Loss: 0.04578676074743271
Epoch 17403/30000 Training Loss: 0.04300684109330177
Epoch 17404/30000 Training Loss: 0.048017822206020355
Epoch 17405/30000 Training Loss: 0.050269853323698044
Epoch 17406/30000 Training Loss: 0.04

Epoch 17540/30000 Training Loss: 0.03988983854651451
Epoch 17541/30000 Training Loss: 0.043158747255802155
Epoch 17542/30000 Training Loss: 0.04662735387682915
Epoch 17543/30000 Training Loss: 0.04229269176721573
Epoch 17544/30000 Training Loss: 0.045148495584726334
Epoch 17545/30000 Training Loss: 0.049619805067777634
Epoch 17546/30000 Training Loss: 0.045305781066417694
Epoch 17547/30000 Training Loss: 0.04603513702750206
Epoch 17548/30000 Training Loss: 0.04339087754487991
Epoch 17549/30000 Training Loss: 0.04275808483362198
Epoch 17550/30000 Training Loss: 0.045913778245449066
Epoch 17550/30000 Validation Loss: 0.04354684799909592
Epoch 17551/30000 Training Loss: 0.0428299717605114
Epoch 17552/30000 Training Loss: 0.04986891150474548
Epoch 17553/30000 Training Loss: 0.04496791213750839
Epoch 17554/30000 Training Loss: 0.04697640985250473
Epoch 17555/30000 Training Loss: 0.04008740931749344
Epoch 17556/30000 Training Loss: 0.04345877468585968
Epoch 17557/30000 Training Loss: 0.04015

In [ ]:
def plot(train_losses, val_losses):

    title_size = 28
    label_size = 21
    legend_size = 18
    tick_size = 17
    
    plot_folder_path = "plots"
    if not os.path.exists(plot_folder_path):
        os.makedirs(plot_folder_path)
        
    sns.set(style="whitegrid")
    fig, ax = plt.subplots(figsize=(10, 7))  
    train_epochs = np.arange(1, len(train_losses)+1)
    val_epochs = np.arange(0, len(train_losses)+1, int(len(train_losses)/(len(val_losses)-1)))
    val_epochs[0] = 1
    
    ax.plot(train_epochs, train_losses, 'tab:blue', label="Training Loss")
    ax.plot(val_epochs, val_losses, 'tab:orange', label=f"Validation Loss (at every {int(len(train_losses)/(len(val_losses)-1))} interval)")
    ax.tick_params(labelsize=tick_size)
    ax.set_title('Loss', fontsize=title_size)
    ax.set_xlabel('Epochs', fontsize=label_size)
    ax.set_ylabel('Loss', fontsize=label_size)
    ax.legend(loc="upper right", fontsize=legend_size)
    

    plt.tight_layout()
    plt.savefig(os.path.join(plot_folder_path, "loss_plot.png"))

In [ ]:
plot(train_loss, val_loss)

In [ ]:
# param_names = []
# for i, (name, param) in enumerate(model.named_parameters()):
#     param_names.append(name)
#     print(f'{i}: {name} {param.requires_grad}')

In [ ]:
# dt = datetime.now().strftime("%m%d")
# best_model_fp = f'models/0221/model_best_epoch_20600_min_val_loss_0.028699999675154686.pkl'
# with open(best_model_fp, 'rb') as handle:
#     best_model = pickle.load(handle)

# Generating sequences

In [ ]:
_, diffusion = create_model_and_diffusion(
                        hidden_t_dim,
                        hidden_dim,
                        vocab_size,
                        config_name,
                        use_plm_init,
                        dropout,
                        2000,
                        noise_schedule,
                        predict_xstart,
                        rescale_timesteps,
                    )

In [ ]:
word_lst_source, word_lst_recover, word_lst_ref = sampling(model, 
                                                           diffusion, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=128)

Generating 20 sentences takes 5 minutes

In [ ]:
word_lst_source

In [ ]:
word_lst_recover

In [ ]:
word_lst_ref